# Домашнее задание № 3

**Предметная область:** медицинская клиника.

**Дано:**
* файл с результатами анализов в зашифрованном виде - medicine.xlsx (файл располагается в одном каталоге с итоговой программой).\
* таблицы med_an_name, med_name для расшифровки показаний (в рамках задания создайте их в своей базе данных, код ниже). 

**Что необходимо сделать:**
1) Режим попроще:
    * забираете данные с листа `easy`
    * вычисляете пациентов, у которых не в порядке хотябы один анализ
    * сохраняете в xlsx имя, телефон, название анализа и заключение: 'Повышен' или 'Понижен'

2) Режим посложнее:
    * забираете данные с листа `hard`
    * вычисляете пациентов, у которых не в порядке два и более анализов
    * сохраняете в xlsx имя, телефон, название анализа и заключение: 'Повышен', 'Понижен' или 'Положительный'

**Что ожидается на выходе:**\
В ответе приложите файл - скрипт python (название easy.py или hard.py в зависимости от выбранного режима)\
Результатом работы скрипта должен быть xlsx файл с заданной информацией.

#### *Создание таблицы с медицинскими показателями*

#### *Создание таблицы с информацией о клиентах*

#### *Решение*

#### 1. Загрузка данных с таблицы PostgreSQL 

In [1]:
import psycopg2

# Подключение к базе данных
conn = psycopg2.connect(
    database="postgres",
    host="localhost",
    user="postgres",
    password="postgres",
    port="5433"
)

conn.autocommit = False
cur = conn.cursor()

# Удаление таблиц
drop_tables = """
DROP TABLE IF EXISTS med_an_name;
DROP TABLE IF EXISTS med_name;
"""

# Создание таблиц
create_med_an_name = """
CREATE TABLE med_an_name (
	id varchar(10) PRIMARY KEY,
	name varchar(200) NULL,
	is_simple bpchar(1) NULL,
	min_value numeric(18, 2) NULL,
	max_value numeric(18, 2) NULL
);
"""

create_med_name = """
CREATE TABLE med_name (
	id int4 PRIMARY KEY,
	name varchar(50) NULL,
	phone varchar(20) NULL
);
"""

# Данные для med_an_name
insert_med_an_name = """
INSERT INTO med_an_name (id, name, is_simple, min_value, max_value) VALUES
	 ('IG','Иммуноглобулин общий','N',0.00,100.00),
	 ('ALAT','Аланинаминотрансфераза ','N',0.00,45.00),
	 ('2-A','Лейкоциты (моча)','N',0.00,5.00),
	 ('G124','Гаптоглобин','N',150.00,2000.00),
	 ('890','Глюкоза','N',3.30,5.50),
	 ('S','Щелочная фосфатаза','N',40.00,150.00),
	 ('1-100','Общий белок ','N',64.00,84.00),
	 ('N','Нитрит (моча)','Y',NULL,NULL),
	 ('3-511','Креатинин','N',62.00,115.00),
	 ('Z','Эритроциты (моча)','N',0.00,2.00),
	 ('IRR','Липаза','N',0.00,190.00),
	 ('GLK','Глюкоза (моча)','Y',NULL,NULL),
	 ('1-875','Грибки (моча)','Y',NULL,NULL),
	 ('1-900','Амилаза панкреатическая','N',0.00,50.00),
	 ('BBB','Белок (моча)','Y',NULL,NULL),
	 ('AU','Гематокрит','N',39.00,49.00),
	 ('C','СОЭ','N',2.00,20.00),
	 ('aZz','Билирубин','N',5.00,20.00);
"""

# Данные для med_name
insert_med_name = """
INSERT INTO med_name (id, name, phone) VALUES
	 (191,'Чемиренко Д.А','+7 (905) 912-28-47'),
	 (140,'Головцов Р.Р','+7 (906) 949-98-17'),
	 (119,'Сакобов А.Т','+7 (952) 996-60-21'),
	 (57,'Тарлов И.Е','+7 (900) 875-67-38'),
	 (195,'Свенюков Б.Ю','+7 (983) 577-27-21'),
	 (53,'Попелицкий Р.Г','+7 (976) 841-79-95'),
	 (96,'Умралиев Р.Э','+7 (941) 954-55-88'),
	 (62,'Бондарев С.Г','+7 (906) 933-95-63'),
	 (199,'Цетнарский Э.И','+7 (908) 245-80-47'),
	 (114,'Даклеев Т.М','+7 (925) 612-93-95'),
	 (73,'Гармокацкий С.Л','+7 (935) 653-32-78'),
	 (80,'Маржецкий А.Я','+7 (949) 912-97-84'),
	 (151,'Поляховский Г.Д','+7 (998) 596-45-47'),
	 (127,'Галендук Л.Г','+7 (935) 164-87-46'),
	 (135,'Номоконов Г.Р','+7 (940) 384-74-90'),
	 (77,'Шелкоплясов Э.В','+7 (979) 643-71-75'),
	 (67,'Эльменькин Д.Р','+7 (982) 643-36-73'),
	 (176,'Ламок А.О','+7 (984) 779-54-57'),
	 (163,'Бочечкаров Э.Д','+7 (943) 102-79-69'),
	 (192,'Голяминских Г.Р','+7 (903) 280-60-58'),
	 (72,'Пахарев Ю.Э','+7 (977) 810-48-75'),
	 (111,'Ведмидев Я.У','+7 (997) 541-22-17'),
	 (118,'Сисемкин В.Р','+7 (960) 542-96-83'),
	 (148,'Штыкулин И.А','+7 (949) 862-47-83'),
	 (162,'Шапаев Л.А','+7 (919) 109-43-56'),
	 (76,'Млицкий С.А','+7 (987) 542-73-38'),
	 (101,'Загатин А.Д','+7 (987) 928-64-89');
"""

try:
    cur.execute(drop_tables)
    cur.execute(create_med_an_name)
    cur.execute(create_med_name)

    cur.execute(insert_med_an_name)
    cur.execute(insert_med_name)

    conn.commit()
    print("Все таблицы успешно созданы и заполнены.")
except Exception as e:
    conn.rollback()
    print(f"Ошибка: {e}")
finally:
    cur.close()
    conn.close()

Все таблицы успешно созданы и заполнены.


#### Режим попроще:

- забираете данные с листа easy
- вычисляете пациентов, у которых не в порядке хотя бы один анализ
- сохраняете в xlsx имя, телефон, название анализа и заключение: 'Повышен' или 'Понижен'

In [2]:
import pandas as pd
import psycopg2


def get_sql(ps_cursor, sql_text):
    # Выполнение SQL-запроса
    ps_cursor.execute(sql_text)
    # получение списка кортежей результата селекта
    results = ps_cursor.fetchall()
    col_names = [row[0] for row in ps_cursor.description]
    print(f'По запросу: "{sql_text}" получено {ps_cursor.rowcount} записей\n')

    # полученные данные преобразуем в датафрейм
    df = pd.DataFrame(data=results, columns=col_names)

    return df.fillna(0)


# Подключение к базе данных
connection = psycopg2.connect(
    database="postgres",
    host="localhost",
    user="postgres",
    password="postgres",
    port="5433"
)

# Курсор для выполнения операций с базой данных
cursor = connection.cursor()

sql_med_name = 'select n.id "Код пациента", n.name "ФИО", n.phone "Телефон" from med_name n'
sql_med_an_name = ('select m.id "Анализ", m.name "Название анализа", m.is_simple, '
                   'm.min_value, m.max_value from med_an_name m')

med_name = get_sql(cursor, sql_med_name)
med_an_name = get_sql(cursor, sql_med_an_name)

# после выполнения всех действий закрываем соединение с базой
cursor.close()
connection.close()

data = pd.read_excel('medicine.xlsx', sheet_name='easy', converters={'Анализ': str})

# Объединяем датафреймы
data = (data
        .merge(med_name, on='Код пациента', how='left')
        .merge(med_an_name, on='Анализ', how='left')
        .sort_values(['ФИО', 'Анализ'])
        .reset_index(drop=True)
        )
# Строим новую колонку с отклонениями
data['outlet'] = data.apply(lambda row: (row['Значение'] > row.max_value) -
                                        (row['Значение'] < row.min_value), axis=1)
# Фильтруем строки с отклонениями
easy = data[data['outlet'] != 0].copy()
# Формируем Заключение в нужном формате
easy['Заключение'] = easy['outlet'].map({-1: 'Понижен', 1: 'Повышен'})
# Сохраняем в файл
save_columns = ['ФИО', 'Телефон', 'Название анализа', 'Заключение']
easy[save_columns].to_excel('easy.xlsx', index=False, sheet_name='Результат')

print(easy[save_columns])

По запросу: "select n.id "Код пациента", n.name "ФИО", n.phone "Телефон" from med_name n" получено 27 записей

По запросу: "select m.id "Анализ", m.name "Название анализа", m.is_simple, m.min_value, m.max_value from med_an_name m" получено 18 записей

                ФИО             Телефон         Название анализа Заключение
5   Гармокацкий С.Л  +7 (935) 653-32-78       Щелочная фосфатаза    Понижен
6      Головцов Р.Р  +7 (906) 949-98-17  Аланинаминотрансфераза     Повышен
17   Попелицкий Р.Г  +7 (976) 841-79-95       Щелочная фосфатаза    Понижен
22     Умралиев Р.Э  +7 (941) 954-55-88             Общий белок     Повышен


#### Режим посложнее:

- забираете данные с листа hard
- вычисляете пациентов, у которых не в порядке два и более анализов
- сохраняете в xlsx имя, телефон, название анализа и заключение: 'Повышен', 'Понижен' или 'Положительный'

In [3]:
import pandas as pd
import psycopg2


def get_sql(ps_cursor, sql_text):
    # Выполнение SQL-запроса
    ps_cursor.execute(sql_text)
    # получение списка кортежей результата селекта
    results = ps_cursor.fetchall()
    col_names = [row[0] for row in ps_cursor.description]
    print(f'По запросу: "{sql_text}" получено {ps_cursor.rowcount} записей\n')

    # полученные данные преобразуем в датафрейм
    df = pd.DataFrame(data=results, columns=col_names)

    return df.fillna(0)


# Подключение к базе данных
connection = psycopg2.connect(
    database="postgres",
    host="localhost",
    user="postgres",
    password="postgres",
    port="5433"
)

# Курсор для выполнения операций с базой данных
cursor = connection.cursor()

sql_med_name = 'select n.id "Код пациента", n.name "ФИО", n.phone "Телефон" from med_name n'
sql_med_an_name = ('select m.id "Анализ", m.name "Название анализа", m.is_simple, '
                   'm.min_value, m.max_value from med_an_name m')

med_name = get_sql(cursor, sql_med_name)
med_an_name = get_sql(cursor, sql_med_an_name)

# после выполнения всех действий закрываем соединение с базой
cursor.close()
connection.close()

data = pd.read_excel('medicine.xlsx', sheet_name='hard', converters={'Анализ': str})

# Объединяем датафреймы
data = (data
        .merge(med_name, on='Код пациента', how='left')
        .merge(med_an_name, on='Анализ', how='left')
        .sort_values(['ФИО', 'Анализ'])
        .reset_index(drop=True)
        )
# Приводим положительные значения бинарных результатов к единым значениям
data['Значение'] = data['Значение'].map(lambda z: 'Положит.' if z == '+' else z)

# Строим новую колонку с численными значениями: строки будут NaN
data['value'] = pd.to_numeric(data['Значение'], errors='coerce', downcast='float')

# Строим новую колонку с отклонениями
data['outlet'] = data.apply(
    lambda row: (row['value'] > row.max_value) - (row['value'] < row.min_value), axis=1)

# Заполняем отклонения для бинарных результатов с положительным значением
data.loc[data['Значение'].astype(str).str.contains('Полож'), 'outlet'] = 2

# Фильтруем строки с отклонениями
hard = data[data['outlet'] != 0].copy()

# Выбираем ФИО, у которых есть отклонения в двух и более анализах
fio_outlet = hard['ФИО'].value_counts()
fio_outlet = fio_outlet[fio_outlet > 1].index.tolist()

# Фильтруем строки с найденными ФИО
hard = hard[hard['ФИО'].isin(fio_outlet)].copy()

# Формируем Заключение в нужном формате
hard['Заключение'] = hard['outlet'].map({-1: 'Понижен', 1: 'Повышен', 2: 'Положительный'})

# Сохраняем в файл
save_columns = ['ФИО', 'Телефон', 'Название анализа', 'Заключение']
hard[save_columns].to_excel('hard.xlsx', index=False, sheet_name='Результат')

print(hard[save_columns])

По запросу: "select n.id "Код пациента", n.name "ФИО", n.phone "Телефон" from med_name n" получено 27 записей

По запросу: "select m.id "Анализ", m.name "Название анализа", m.is_simple, m.min_value, m.max_value from med_an_name m" получено 18 записей

               ФИО             Телефон    Название анализа     Заключение
6  Гармокацкий С.Л  +7 (935) 653-32-78       Нитрит (моча)  Положительный
7  Гармокацкий С.Л  +7 (935) 653-32-78  Щелочная фосфатаза        Понижен
